In [1]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("Device:", torch.device("cuda" if torch.cuda.is_available() else "cpu"))

CUDA available: True
Device: cuda


In [2]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import numpy as np
import rasterio
import albumentations as A
import timm
from tqdm import tqdm
import csv
from torch.cuda.amp import autocast, GradScaler

In [3]:
class CloudDataset(Dataset):
    def __init__(self, metadata, root_dir, transform=None):
        self.metadata = metadata
        self.root_dir = root_dir
        self.transform = transform
        self.feature_dir = os.path.join(root_dir, 'train_features')
        self.label_dir = os.path.join(root_dir, 'train_labels')

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):
        chip_id = self.metadata.iloc[idx]['chip_id']
        feature_path = os.path.join(self.feature_dir, chip_id)
        label_path = os.path.join(self.label_dir, f"{chip_id}.tif")

        bands = ['B02', 'B03', 'B04', 'B08']
        features = []

        for band in bands:
            with rasterio.open(os.path.join(feature_path, f'{band}.tif')) as src:
                features.append(src.read(1).astype(np.float32))

        image = np.stack(features, axis=-1)

        with rasterio.open(label_path) as src:
            mask = src.read(1).astype(np.float32)

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        image = torch.from_numpy(image).permute(2, 0, 1).float()
        mask = torch.from_numpy(mask).unsqueeze(0).float()

        return image, mask

In [4]:
mean_vals = (0.5, 0.5, 0.5, 0.5)
std_vals = (0.5, 0.5, 0.5, 0.5)

train_transform = A.Compose([
    A.Resize(256, 256),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.Normalize(mean=mean_vals, std=std_vals, max_pixel_value=65535.0),
], additional_targets={'mask': 'mask'})

val_transform = A.Compose([
    A.Resize(256, 256),
    A.Normalize(mean=mean_vals, std=std_vals, max_pixel_value=65535.0),
], additional_targets={'mask': 'mask'})


In [5]:
root_dir = 'data'
metadata = pd.read_csv('data/train_metadata.csv')

train_meta, temp_meta = train_test_split(metadata, test_size=0.3, random_state=42)
val_meta, test_meta = train_test_split(temp_meta, test_size=2/3, random_state=42)


In [6]:
train_dataset = CloudDataset(train_meta, root_dir, train_transform)
val_dataset   = CloudDataset(val_meta, root_dir, val_transform)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=0)
val_loader   = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=0)

print("Train samples:", len(train_dataset))
print("Val samples:", len(val_dataset))


Train samples: 8223
Val samples: 1175


In [7]:
class XceptionBackbone(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = timm.create_model(
            'xception41', pretrained=True, features_only=True, in_chans=4
        )

    def forward(self, x):
        features = self.model(x)
        return features[1], features[3]
class ASPP(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU()
        )

    def forward(self, x):
        return self.conv(x)
class ImprovedDeepLabV3Plus(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = XceptionBackbone()
        self.aspp = ASPP(1024, 256)
        self.final = nn.Conv2d(256, 1, 1)

    def forward(self, x):
        h, w = x.shape[-2:]
        low, high = self.backbone(x)
        x = self.aspp(high)
        x = F.interpolate(x, size=(h, w), mode='bilinear', align_corners=False)
        return torch.sigmoid(self.final(x))


In [8]:
class DiceLoss(nn.Module):
    def forward(self, pred, target):
        smooth = 1.
        pred = pred.view(-1)
        target = target.view(-1)
        intersection = (pred * target).sum()
        return 1 - (2.*intersection + smooth)/(pred.sum()+target.sum()+smooth)
def compute_iou(pred, target):
    pred = (pred > 0.5).float()
    intersection = (pred * target).sum()
    union = pred.sum() + target.sum() - intersection
    return (intersection / (union + 1e-6)).item()

In [9]:
def compute_metrics(pred, target, threshold=0.5):
    pred = (pred > threshold).float()

    TP = (pred * target).sum()
    TN = ((1 - pred) * (1 - target)).sum()
    FP = (pred * (1 - target)).sum()
    FN = ((1 - pred) * target).sum()

    accuracy = (TP + TN) / (TP + TN + FP + FN + 1e-6)
    precision = TP / (TP + FP + 1e-6)
    recall = TP / (TP + FN + 1e-6)
    iou = TP / (TP + FP + FN + 1e-6)

    return accuracy.item(), precision.item(), recall.item(), iou.item()


In [10]:
from torch.amp import autocast, GradScaler
import csv

def train_model(
    model,
    train_loader,
    val_loader,
    num_epochs=35,
    patience=35
):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50)

    # ✅ AMP-SAFE LOSS
    bce_loss = nn.BCEWithLogitsLoss()
    criterion = lambda p, t: (
        0.5 * bce_loss(p, t) +
        0.5 * DiceLoss()(torch.sigmoid(p), t)
    )

    scaler = GradScaler("cuda")
    best_val_loss = float('inf')
    epochs_no_improve = 0

    with open("training_results3.csv", "w", newline="") as f:
        writer = csv.DictWriter(
            f,
            fieldnames=[
                "epoch", "train_loss", "val_loss",
                "accuracy", "precision", "recall", "iou"
            ]
        )
        writer.writeheader()

        for epoch in range(num_epochs):
            # ---------- TRAIN ----------
            model.train()
            train_loss = 0.0
            acc = prec = rec = iou = 0.0

            for imgs, masks in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
                imgs, masks = imgs.to(device), masks.to(device)
                optimizer.zero_grad()

                with autocast(device_type="cuda"):
                    preds = model(imgs)          # logits
                    loss = criterion(preds, masks)

                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()

                train_loss += loss.item()

                # Metrics use probabilities
                probs = torch.sigmoid(preds)
                a, p, r, i = compute_metrics(probs, masks)
                acc += a; prec += p; rec += r; iou += i

            train_loss /= len(train_loader)
            acc /= len(train_loader)
            prec /= len(train_loader)
            rec /= len(train_loader)
            iou /= len(train_loader)

            # ---------- VALIDATION ----------
            model.eval()
            val_loss = 0.0
            with torch.no_grad():
                for imgs, masks in val_loader:
                    imgs, masks = imgs.to(device), masks.to(device)
                    preds = model(imgs)
                    val_loss += criterion(preds, masks).item()

            val_loss /= len(val_loader)

            writer.writerow({
                "epoch": epoch + 1,
                "train_loss": train_loss,
                "val_loss": val_loss,
                "accuracy": acc,
                "precision": prec,
                "recall": rec,
                "iou": iou
            })
            f.flush()

            scheduler.step()

            print(
                f"Epoch {epoch+1}: "
                f"Train={train_loss:.4f}, Val={val_loss:.4f}, IoU={iou:.4f}"
            )

            # ---------- EARLY STOPPING ----------
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                epochs_no_improve = 0
                torch.save(model.state_dict(), "final1_model3.pth")
            else:
                epochs_no_improve += 1
                if epochs_no_improve >= patience:
                    print("⛔ Early stopping triggered")
                    break

In [ ]:
model = ImprovedDeepLabV3Plus()
train_model(model, train_loader, val_loader, num_epochs=35)

Epoch 1/35: 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 1028/1028 [07:23<00:00,  2.32it/s]


Epoch 1: Train=0.4388, Val=0.4082, IoU=0.6790


Epoch 2/35: 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 1028/1028 [07:44<00:00,  2.21it/s]


Epoch 2: Train=0.4288, Val=0.4050, IoU=0.7220


Epoch 3/35: 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 1028/1028 [07:33<00:00,  2.27it/s]


Epoch 3: Train=0.4285, Val=0.4115, IoU=0.7463


Epoch 4/35: 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 1028/1028 [07:28<00:00,  2.29it/s]


Epoch 4: Train=0.4258, Val=0.4046, IoU=0.7649


Epoch 5/35:  22%|█████████████████████▎                                                                         | 231/1028 [01:42<06:00,  2.21it/s]

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("training_results3.csv")

# ---------- LOSS PLOT ----------
plt.figure(figsize=(8,5))
plt.plot(df["epoch"], df["train_loss"], label="Train Loss")
plt.plot(df["epoch"], df["val_loss"], label="Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training vs Validation Loss")
plt.legend()
plt.grid(True)
plt.show()

# ---------- METRICS PLOT ----------
plt.figure(figsize=(8,5))
plt.plot(df["epoch"], df["accuracy"], label="Accuracy")
plt.plot(df["epoch"], df["precision"], label="Precision")
plt.plot(df["epoch"], df["recall"], label="Recall")
plt.plot(df["epoch"], df["iou"], label="IoU")
plt.xlabel("Epoch")
plt.ylabel("Score")
plt.title("Segmentation Metrics vs Epoch")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
import random

num_images = 3
indices = random.sample(range(len(val_dataset)), num_images)

with torch.no_grad():
    for idx in indices:
        image, mask = val_dataset[idx]

        image = image.unsqueeze(0).to(device)
        mask = mask.unsqueeze(0).to(device)

        output = model(image)
        pred = (output > 0.5).float()

        img = image[0].cpu().numpy()
        gt = mask[0][0].cpu().numpy()
        pr = pred[0][0].cpu().numpy()

        img = (img - img.min()) / (img.max() - img.min() + 1e-8)
        rgb = img[[2, 1, 0], :, :].transpose(1, 2, 0)

        fig, axs = plt.subplots(1, 3, figsize=(15, 5))
        axs[0].imshow(rgb)
        axs[0].set_title("RGB Image")

        axs[1].imshow(gt, cmap="gray")
        axs[1].set_title("Ground Truth")

        axs[2].imshow(pr, cmap="gray")
        axs[2].set_title("Prediction")

        for ax in axs:
            ax.axis("off")

        plt.tight_layout()
        plt.show()
